## End-to-End Virtual Screening Pipeline with BioNeMo
This example notebook shows how to connect BioNeMo service models to conduct a virtual drug screening workflow. We'll infuse our workflow with AI at every step, from ligand generation to protein folding to docking. All of these capabilities are enabled by NVIDIA's BioNeMo framework for Large Language Model training and inference. For more details, please visit the NVIDIA BioNeMo Service homepage at https://www.nvidia.com/en-us/gpu-cloud/bionemo/

Let's break down the key steps of a virtual screening workflow, and map each component to the AI model capabilities enabled in BioNeMo. 

<img src="images/virtual-screening-workflow.jpg" title="Virtual Screening Workflow">

#### Protein Structure Prediction
Protein structure prediction refers to predicting the three-dimensional structure of a protein based on its amino acid sequence. This is important because a protein’s function is often determined by its structure, and accurate structure prediction can aid in drug discovery. Protein structure prediction models (such as AlphaFold2, ESMFold and OpenFold) are typically trained using deep learning techniques on large datasets of known protein structures. These models have many uses, including drug discovery, understanding disease mechanisms, and designing new proteins with specific functions. A high-quality protein structure provides researchers with a an authentic biological target for drug design.

#### Small Molecule Generation
Small molecule generation is the process of using deep learning models to generate novel chemical structures. These models are typically trained on large databases of existing molecules and learn to generate new molecules that have similar properties to those in the training set. Small molecule generation models, such as MegaMolBART and MoFlow can be used for a variety of applications, including drug discovery, that involve in silico design of novel molecules. They allow for the generation of novel compounds and can aid in the discovery of new drug candidates with improved efficacy, safety, and specificity.

#### Molecular Docking
Molecular docking is a computational method used to predict the binding affinity and structure of a small molecule ligand binding to a protein target. This information allows for the identification of potential drug candidates that can selectively bind to a target protein and modulate its function. Classical molecular docking methods are often based on simulations or heuristics such as complementary shapes. However, these can be either computationally expensive, or rely on approximations and simplifications that can limit their accuracy. Machine learning models such as DiffDock offer new solutions for molecular docking by leveraging existing data to provide more accurate and efficient methods for predicting protein-ligand interactions. DiffDock learns the complex structural patterns of known ligand-bound proteins (holo structures), uses this learning to predict multiple binding poses for a given ligand-protein pair and ranks the poses in order of prediction confidence.

Now that we've clarified the major steps of the drug discovery process, let's get started with a hands-on example. 
To begin, please configure and provide your NGC access token by visiting https://ngc.nvidia.com/setup/api-key

### BioNeMo Service Configurations

In [1]:
API_KEY="YOUR KEY HERE"
API_HOST="https://api.bionemo.ngc.nvidia.com/v1"

Let's start by installing and importing library dependencies. We'll use _requests_ to interact with the BioNeMo service, _pandas_ and _rdkit_ to manage and manipulate our chemical data, and _py3Dmol_ for molecule visualization.

In [ ]:
!pip install requests pandas rdkit py3dmol

In [ ]:
import requests
import json
import os
import time
import py3Dmol
import ast
import pandas as pd

# Importing libraries from RD-Kit for handling and manipulating chemical data
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

Next let's validate our connection to the BioNeMo service. You should get a "200" HTTP response in the following cell, indicating a successful connection. Note the _Authorization_ field in the header response which contains your NGC access token credentials. We'll re-use this header for all future interactions with the service.
<div class="alert alert-block alert-info">
    <b>Tip:</b>  If you do not receive a "200 OK" HTTP response when testing connectivity to BioNeMo service in the cell below, verify
    
 - the API_HOST defined above is the correct address of the BioNeMo service, and
 - the API_KEY is authorized to access BioNeMo service at the API_HOST

Refer to this <a href="https://developer.mozilla.org/en-US/docs/Web/HTTP/Status">list of HTTP responses status codes</a> to help determine the cause of a non-200 response.
</div>

In [ ]:
#Checking to see if the access is configured
response = requests.get(
    f"{API_HOST}/models",
    headers={"Authorization": f"Bearer {API_KEY}"})
print("Query BioNeMo Service:", response)

#Add key to headers for remainder of notebook
headers = {
    'Authorization': f'Bearer {API_KEY}'
}

Some of the BioNeMo services such as are _sychronous_, meaning the service API call will block until a result is returned. Two examples of synchronous services are MegaMolBART molecule embedding, and looking up PDBs using the BioNeMo Uniprot service.Other services are _asynchronous_, such as the protein folding and docking services. Functions calling _asynchronous_ services are nonblocking, and immediately return a handle called a _correlation_id_ that can be used to query the results. This allows us to batch multiple requests together and query them for completion at a later time.

In the following cell we introduce a helper function _query_async_result_ that will block on a _correlation_id_ task until the computation is completely and a result is returned.

In [ ]:
def query_async_result(request, print_result=False):
    if isinstance(request, str):
        #Request is a correlation id string
        correlation_id=request
    elif isinstance(request, requests.models.Response):
        submission_response = json.loads(request.content)
        correlation_id=submission_response['correlation_id']
    
    i = 0
    while True:
        response = requests.get(
            f"{API_HOST}/task/{correlation_id}",
            headers=headers,
        )      
    
        status_result = json.loads(response.content)
        if status_result['control_info']['status'] == 'DONE':
            if(print_result): print(status_result['response'])
            return(status_result['response'])
        if status_result['control_info']['status'] == 'ERROR':
            print("ERROR, Cancelling Result Retrieval")
            return            
        else:
            print(f"{status_result['control_info']['status']}{''.join(['.']*i)}" , end="\r")
            time.sleep(10)  # waiting for the prediction from BioNeMo Server
        i = i+1

## Dihydrofolate reductase (DHFR)

Dihydrofolate reductase (DHFR) is a critical enzyme involved in the folate metabolic pathway, which is responsible for the synthesis of nucleotides, amino acids, and other essential cellular components. DHFR catalyzes the conversion of dihydrofolate (DHF) to tetrahydrofolate (THF), which is required for the synthesis of purines, pyrimidines, and other molecules necessary for DNA synthesis and cellular proliferation.

DHFR inhibitors have been used as anticancer and antibacterial agents due to their ability to disrupt the folate metabolic pathway and inhibit DNA synthesis. Methotrexate, a potent inhibitor of DHFR, has been used to treat several types of cancer, including leukemia and lymphoma. Trimethoprim, another DHFR inhibitor, has been used as an antibacterial agent to treat bacterial infections.

In this notebook, we take a virtual drug screening approach used to identify potential drug candidates that can bind to a the DHFR protein found in Humans. In this approach, we will use AI generative models to construct a library of small molecules that are similar in structure to known DHFR inhibitors.  After obtaining a DHFR protein sequence and folding it with an AI model - we'll use a diffusion generative model to predict docked binding poses. In a production workflow, this data would be used to predict candidates with the strongest binding affinities to DHFR. The most promising targets would be synthesized experimentally to confirm their activity and selectivity towards the target protein.

Before we start generating drug candidates, let's begin by retrieving the amino acid sequence for DHFR and predicting its folded structure! Given a UniProt protein ID, the BioNeMo service can be used to retrieve the corresponding amino acid sequence.

In [ ]:
uniprot_id="P00374"
response = requests.get(f'{API_HOST}/uniprot/{uniprot_id}', headers=headers)
sequence = json.loads(response.content)
print(sequence)

## Folding DHFR with OpenFold
With the sequence in hand, we're ready to predict a folded structure. Note the reference to `/protein-structure/openfold/predict` in the service submission message. You can modify the endpoint to select different functionally equivalent models.

<div class="alert alert-block alert-info">
    <b>Tip:</b>  Feel free to come back to this cell and change the <code>folding_model</code> variable from <code>openfold</code> to <code>alphafold2</code>. The best model will depend on the particular sequence being folded. A great benefit of the BioNeMo service is having a sandbox to mix-and-match models and compare results! 
</div>

We'll save the pdb data in an actual file, as the difficusion models we're about to use require a file-based input. 

In [ ]:
#Choose 'openfold' or 'alphafold2'
folding_model='openfold'

submission_request = requests.post(
    f"{API_HOST}/protein-structure/openfold/predict",
    headers=headers,
    files={
        "sequence": (None, sequence),
    }
)

result=query_async_result(submission_request)

pdb_filename = f"BioNeMo_{folding_model}_DHFR.pdb"
with open(pdb_filename, 'w') as pdb_file:
    for line in json.loads(result)['pdbs']:
        pdb_file.write(line)

Let's check the protein structure for a quick sanity check.

In [ ]:
# Visualize the predicted structure saved in PDB file
with open(pdb_filename) as ifile:
    system = "".join([x for x in ifile])
#configuring the structure display
view = py3Dmol.view(width=600, height=400)
view.addModelsAsFrames(system)
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()

## Generating Drug Conformers with MoFlow

Now that we have a folded protein structure, it's time to generate some ligand targets for docking. We will start with an existing dataset of known DHFR inhibitors provided by PubChem BioAssays repository.

https://pubchem.ncbi.nlm.nih.gov/bioassay/56812#section=Data-Table

The data table for this BioAssay contains 12 compounds that are analogues of folic acid and aminopterin. Aminopterin was first developed as an anticancer agent in the 1940s and later used as a model compound for the development of methotrexate, which is a very relevant DHFR inhibitor in the clinic today. The _csv_ data for the table was downloaded and is included in the present notebook directory. After reading the data into a Pandas dataframe and extracting the SMILES strings, we can visualize the molecules with RDKit.

In [ ]:
df = pd.read_csv('data/AID_56812_datatable_all.csv')
smiles_column_name = 'PUBCHEM_EXT_DATASOURCE_SMILES'

smis=[]
for index, row in df.iterrows():
    if not pd.isna(row[smiles_column_name]):
        smis.append(row[smiles_column_name])

print("Number of unique molecule designs collected from bioassay: ", len(smis))
mols_from_assay = [Chem.MolFromSmiles(smi) for smi in smis]
Draw.MolsToGridImage(mols_from_assay, molsPerRow=4, subImgSize=(300,300))

Next we're going to use these known drug molecules to seed a deep graph generative model called MoFlow. Using the BioNeMo service, the model will create molecules with similar chemical structure to the seed compounds.  Since MoFlow is an _asynchronous_ service, it would be most efficient to batch and send all service requests at once before we start querying results.

In [ ]:
#Batch requests before querying for any results
correlation_ids=[]
for smi in smis:
    submission_request = requests.post(
        f"{API_HOST}/molecule/moflow/generate",
        headers=headers,
        json={
            "smi": smi,
            "num_samples":20
        }
    )
    
    submission_response = json.loads(submission_request.content)
    correlation_ids.append(submission_response['correlation_id'])
print(correlation_ids)

We'll start querying the results one-at-a-time. Since the service request was batched and the seed molecules are similar in size and structure, we can anticipate that the batched responses will finish close together. 

In [ ]:
#Query results
generated_smis=pd.DataFrame()
for correlation_id in correlation_ids:
    result=query_async_result(correlation_id) 
 
    # convert the string to a list of dictionaries
    data_list = ast.literal_eval(result)
    df_dictionary=pd.DataFrame(data_list[0])
    generated_smis = pd.concat([generated_smis, df_dictionary], ignore_index=True)
print(generated_smis)

We now have a reasonable set of compounds to start docking! But even at this early stage, a drug discovery scientist may considering filtering out candidate drugs based on structure or molecular properties. BioNeMo reports a Tanimoto similarity scores for each generated molecule. Let's filter out some of our more exotic candidates that are less likely to exhibit similar bioactivity to our seed molecules. Structures are generally considered similar when their Tanimoto similarity scores are >0.85. We'll start by filtering out similarity scores less than 0.5 so we have plenty of structures to dock with DHFR, but feel free to experiment with the `similarity_threshold` variable. We'll also add the original drugs from the UniProt BioAssay into our drug candidate pool, since we know they should bind strongly. And finish by removing any duplicate structures in our list.

In [ ]:
similarity_threshold=0.5

candidates = []

for index, row in generated_smis.iterrows():
    if(row["similarity_scores"]>=similarity_threshold):
        candidates.append(row["generated_molecules"])


#Add the original drugs from the bioassay        
for smi in smis:
    candidates.append(smi)
    
#Remove any duplicate structures
candidates=list(set(candidates))     
    
print(len(candidates), "Ligand Targets:")
print(candidates)


That brought our list of candidates down considerably, but we still have a reasonable design space to explore. Now that we've narrowed down our list, let's visualize some of the structures with RDKit.

In [ ]:
mols_from_gen_smis = [Chem.MolFromSmiles(smi) for smi in candidates]
print("Number of unique molecule designs: ", len(mols_from_gen_smis))
Draw.MolsToGridImage(mols_from_gen_smis[0:50], molsPerRow=10, subImgSize=(350,350))

Next we'll need to make some changes to our target molecule data to prepare them for docking. The DiffDock service requires that ligand data is provided as _mol2_ or Structure data file _(SDF)_. SDFs contain data for atoms, bonds, and connectivity for ligands. We'll use RDKit to write an SDF for each ligand. Note that the RDKit _AddHs()_ function is also used here to add terminal hydrogen atoms to our ligands. 

In [ ]:
ligand_filenames = []
for index, mol in enumerate(mols_from_gen_smis):
    mols_from_gen_smis[index] = Chem.AddHs(mol)
    filename = f'ligand-{index}.sdf'
    w = Chem.SDWriter(filename)
    w.write(mol)
    w.close()
    ligand_filenames.append(filename)
print("Number of ligand files saved: ", len(ligand_filenames))

## Docking
We've folded our DHFR protein with OpenFold, generated target inhibitor candidates with MoFlow, and saved our data as _pdb_ and _SDF_ files respectfully in preparation for docking. Now it's time to call DiffDock. We'll generate 20 poses for each ligand molecule, and take advantage of the _asynchronous_ nature of the DiffDock service by batching our API requests..

In [ ]:
poses_to_generate=20
poses=pd.DataFrame()
correlation_ids=[]
for index, ligand_filename in enumerate(ligand_filenames):
    files = {
        'ligand_file_bytes': open(ligand_filename, 'rb'),
        'protein_file_bytes': open(pdb_filename, 'rb'),
        'poses_to_generate': (None, poses_to_generate),
        'diffusion_time_divisions': (None, "20"),    
        'diffusion_steps': (None, "18"),    
        'save_diffusion_trajectory': (None, "false")
    }

    submission_request = requests.post(
        f"{API_HOST}/molecular-docking/diffdock/generate",
        headers=headers,
        files=files,
    )
    submission_reponse=json.loads(submission_request.content)
    correlation_ids.append(submission_response['correlation_id'])

print(correlation_ids)

The service requests are sent off. Now to query the responses.

In [ ]:
for correlation_id in correlation_ids:
    print("correlation id:", correlation_id)
    result=query_async_result(submission_request) 
    
     # convert the string to a list of dictionaries
    data_list = ast.literal_eval(result)
    df_dictionary=pd.DataFrame(data_list)
    
    #Add the corresponding ligand SMILES string
    df_dictionary['smiles']=candidates[index]
    poses=pd.concat([poses, df_dictionary], ignore_index=True)

For a sanity check, let's visualize the first pose for our first few drug targets.

In [ ]:
#Read protein data once
with open(pdb_filename, 'r') as f:
    protein_data = f.read()

for index in range(min(3, len(candidates))):

    sdf_data = poses['docked_ligand_position_files'][index*poses_to_generate]

    suppl = Chem.SDMolSupplier()
    suppl.SetData(sdf_data)
    mol = suppl[0]

    view = py3Dmol.view(width=400, height=400)
    
    # Create a Py3Dmol view and add the molecule to it
    view.addModel(Chem.MolToMolBlock(mol), 'sdf')
    view.addModel(protein_data, 'pdb')

    view.setStyle({'model': 0}, {'stick': {}})

    view.zoomTo()
    view.show()

At this point the pose confidence can be used to advise further study. For a high-confidence result, consider sending a service request with a higher number of pose generations. Or use the corresponding ligand SMILES string as a seed for MoFlow or MegaMolBART, itertating on your results to find a stronger target. 

## Conclusion
In this notebook, we've covered a virtual drug screening pipeline using BioNeMo services for each step:
 - Protein structure generation
 - Small molecule generation
 - Molecular Docking
 
Leveraging BioNeMo's capabilities for predicting protein structure and generating a set of target molecules, we are able use docking models to predict multiple binding poses for each given ligand-protein pair and rank the poses in order of prediction confidence.

Use this notebook as a starting guide for your own AI-accelerated virtual drug screening journey with BioNeMo. Now with NVIDIA cloud APIs, researchers can quickly customize and deploy domain-specific, state-of-the-art generative and predictive biomolecular AI models at scale. With these generative AI models, BioNeMo enables researchers to rapidly generate the structure and function of proteins and biomolecules, accelerating the creation of new drug candidates. We can't wait to see what you accomplish!